# Notebook 3: Getting Started with Llama Stack

This notebook will help you set up your environment for this tutorial. Specifically, we will cover installing the necessary libraries, configuring essential parameters, and connecting to a Llama Stack server.

## Installing Dependencies

This code requires `llama-stack` and the `llama-stack-client` python packages. Let's begin by installing them:

In [ ]:
! pip install -qr requirements.txt

## Setting the Environment Variables

We will symlink the [`env.example`](../../../env.example) file to create a new file called `.env`. We've included as many reasonable defaults as possible to get you started, but please use this file to make any customizations needed for your environment such as the the location of the Llama Stack server endpoint..

### Environment variables required for all demos
- `REMOTE_BASE_URL`: the URL of the remote Llama Stack server.
- `TEMPERATURE` (optional): the temperature to use during inference. Defaults to 0.0.
- `TOP_P` (optional): the top_p parameter to use during inference. Defaults to 0.95.
- `MAX_TOKENS` (optional): the maximum number of tokens that can be generated in the completion. Defaults to 512.
- `STREAM` (optional): set this to True to stream the output of the model/agent and False otherwise. Defaults to False.
- `REMOTE_OCP_MCP_URL`: the URL for your Openshift MCP server. If the client does not find the tool registered to the llama-stack instance, it will use this URL to register the Openshift tool.
- `REMOTE_GITHUB_MCP_URL`: the URL for your Github MCP server. If the client does not find the tool registered to the llama-stack instance, it will use this URL to register the Github tool.
- `USE_PROMPT_CHAINING`: dictates if the prompt should be formatted as a few separate prompts to isolate each step or in a single turn.

In [ ]:
!ln -s env.example .env

## Necessary Imports

In [ ]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv(override=True)
from rich import print

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient
from llama_stack_client.types import UserMessage

## Setting Up the Server Connection

Establish the connection to your Llama Stack server.

In [ ]:
base_url = os.getenv("REMOTE_BASE_URL", "http://llamastack-server:8321")
model_id = 'granite-31-2b-instruct'

client = LlamaStackClient(base_url=base_url)

print(base_url)
print(client.models.list())
print(f"Connected to Llama Stack server")

## Initializing the Inference Parameters

Fetch the inference-related parameters from the corresponding environment variables and convert them to the format Llama Stack expects.

In [ ]:
temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "True")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")
print(f"Inference Parameters:\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Now, let's use the Llama stack inference API to greet our LLM. 

In [ ]:
message = UserMessage(
    content="Hi, how are you?",
    role="user",
)
client.inference.chat_completion(
    model_id=model_id,
    messages=[message],
    sampling_params=sampling_params,
    stream=stream
).completion_message.content

# Next

Now that we've connected to Llama Stack, let's get started building the agentic AI system! The next notebook will teach you how to build a [simple agent with tool calling](./4.1_tool_calling.ipynb) application.